# 🎯 FPT Stock Prediction with Market Regime Clustering

## Approach
1. **HMM Clustering (60-day window)**: Phát hiện 4 market regimes
2. **Regime-specific MultiDLinear**: Train model riêng cho mỗi regime
3. **Ensemble Prediction**: Kết hợp predictions theo regime hiện tại
4. **Forecast 100 days**: Tạo submission file

## 1. Setup & Imports

In [ ]:
import os
import random
import warnings
import math
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from hmmlearn import hmm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 2. Load Data & Feature Engineering

In [ ]:
# Load data
df = pd.read_csv('data/FPT_train.csv')
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)

print(f"Shape: {df.shape}")
print(f"Date range: {df['time'].min()} → {df['time'].max()}")
df.head()

In [ ]:
# Feature engineering for model
df['close_log'] = np.log(df['close'])
df['open_log'] = np.log(df['open'])
df['high_log'] = np.log(df['high'])
df['low_log'] = np.log(df['low'])
df['volume_log'] = np.log(df['volume'] + 1)
df['hl_spread'] = (df['high'] - df['low']) / df['close']
df['oc_spread'] = (df['close'] - df['open']) / df['open']
df = df.ffill().bfill()

print("Features created:")
print(df.columns.tolist())

## 3. HMM Market Regime Detection (60-day window)

In [ ]:
# Compute regime features with 60-day rolling window
REGIME_WINDOW = 60

# Return trong window
df['return_60d'] = df['close'].pct_change(REGIME_WINDOW) * 100

# Volatility (annualized)
df['daily_ret'] = df['close'].pct_change()
df['volatility_60d'] = df['daily_ret'].rolling(REGIME_WINDOW).std() * np.sqrt(252) * 100

# Trend strength
def calc_trend(series):
    x = np.arange(len(series))
    slope = np.polyfit(x, series, 1)[0]
    return slope / series.mean() * 100

df['trend_60d'] = df['close'].rolling(REGIME_WINDOW).apply(calc_trend, raw=False)

# Drop NaN rows for regime detection
df_regime = df.dropna(subset=['return_60d', 'volatility_60d', 'trend_60d']).copy()
print(f"Data for regime detection: {len(df_regime)} rows")

In [ ]:
# Fit HMM
N_REGIMES = 4

regime_features = ['return_60d', 'volatility_60d', 'trend_60d']
X_regime = df_regime[regime_features].values

# Scale features
regime_scaler = StandardScaler()
X_regime_scaled = regime_scaler.fit_transform(X_regime)

# Fit HMM
hmm_model = hmm.GaussianHMM(
    n_components=N_REGIMES,
    covariance_type="full",
    n_iter=1000,
    random_state=42
)
hmm_model.fit(X_regime_scaled)

# Predict regimes
df_regime['regime'] = hmm_model.predict(X_regime_scaled)

print("HMM fitted successfully!")
print(f"Transition matrix:\n{hmm_model.transmat_}")

In [ ]:
# Assign meaningful names to regimes
regime_stats = df_regime.groupby('regime').agg({
    'return_60d': 'mean',
    'volatility_60d': 'mean'
})

# Sort by return
sorted_regimes = regime_stats['return_60d'].sort_values(ascending=False).index.tolist()

# Create mapping
REGIME_NAMES = {}
REGIME_COLORS = {}
regime_templates = [
    ('Rally', '#1B5E20'),
    ('Uptrend', '#4CAF50'),
    ('Sideway', '#9E9E9E'),
    ('Downtrend', '#C62828')
]

for i, regime in enumerate(sorted_regimes):
    REGIME_NAMES[regime] = regime_templates[i][0]
    REGIME_COLORS[regime] = regime_templates[i][1]

df_regime['regime_name'] = df_regime['regime'].map(REGIME_NAMES)

# Print summary
print("\n" + "="*60)
print("REGIME SUMMARY")
print("="*60)
for regime in sorted_regimes:
    name = REGIME_NAMES[regime]
    count = (df_regime['regime'] == regime).sum()
    pct = count / len(df_regime) * 100
    ret = regime_stats.loc[regime, 'return_60d']
    vol = regime_stats.loc[regime, 'volatility_60d']
    print(f"\n{name} (Regime {regime}):")
    print(f"  Count: {count} days ({pct:.1f}%)")
    print(f"  Return 60d: {ret:+.1f}%")
    print(f"  Volatility: {vol:.0f}%")

In [ ]:
# Visualize regimes
fig, axes = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

# Price with regime colors
ax = axes[0]
ax.plot(df_regime['time'], df_regime['close'], 'k-', linewidth=1)

# Add regime background
prev_regime = None
start_idx = 0
for i in range(len(df_regime)):
    current = df_regime.iloc[i]['regime']
    if current != prev_regime:
        if prev_regime is not None:
            ax.axvspan(df_regime.iloc[start_idx]['time'],
                      df_regime.iloc[i-1]['time'],
                      alpha=0.3, color=REGIME_COLORS[prev_regime])
        start_idx = i
        prev_regime = current
ax.axvspan(df_regime.iloc[start_idx]['time'],
          df_regime.iloc[-1]['time'],
          alpha=0.3, color=REGIME_COLORS[prev_regime])

ax.set_ylabel('Price (VND)')
ax.set_title('FPT Price with Market Regimes (60-day HMM)', fontweight='bold')

# Legend
import matplotlib.patches as mpatches
legend_elements = [mpatches.Patch(facecolor=REGIME_COLORS[r], alpha=0.5,
                                   label=f"{REGIME_NAMES[r]} ({regime_stats.loc[r, 'return_60d']:+.1f}%)")
                   for r in sorted_regimes]
ax.legend(handles=legend_elements, loc='upper left')

# Regime distribution over time
ax = axes[1]
colors = [REGIME_COLORS[r] for r in df_regime['regime']]
ax.scatter(df_regime['time'], df_regime['regime'], c=colors, s=10, alpha=0.5)
ax.set_yticks(sorted_regimes)
ax.set_yticklabels([REGIME_NAMES[r] for r in sorted_regimes])
ax.set_ylabel('Regime')
ax.set_xlabel('Date')

plt.tight_layout()
plt.show()

## 4. Prepare Data for Model Training

In [ ]:
# Merge regime back to main df
# For rows without regime (first 60 days), use the first available regime
df['regime'] = np.nan
df.loc[df_regime.index, 'regime'] = df_regime['regime'].values
df['regime'] = df['regime'].ffill().bfill().astype(int)

print(f"Regime distribution in full dataset:")
print(df['regime'].value_counts().sort_index())

In [ ]:
# Scaling
train_cutoff = int(len(df) * 0.7)

# Univariate scaler for target
uni_scaler = StandardScaler()
uni_scaler.fit(df['close_log'].values[:train_cutoff].reshape(-1, 1))
df['close_scaled'] = uni_scaler.transform(df['close_log'].values.reshape(-1, 1)).flatten()

# Multivariate scaler for features
multi_cols = ['open_log', 'high_log', 'low_log', 'close_log', 'volume_log', 'hl_spread', 'oc_spread']
multi_scaler = StandardScaler()
multi_scaler.fit(df[multi_cols].values[:train_cutoff])
multi_scaled = multi_scaler.transform(df[multi_cols].values)

n_features = len(multi_cols)
close_idx = 3  # Index of close_log in multi_cols

print(f"Train cutoff: {train_cutoff}")
print(f"N features: {n_features}")

## 5. Dataset & Model Definitions

In [ ]:
class MultivariateDataset(Dataset):
    """Dataset cho multivariate input với regime filtering"""
    def __init__(self, features, target, regimes, seq_len, pred_len, filter_regime=None):
        self.features = features.astype(np.float32)
        self.target = target.astype(np.float32)
        self.regimes = regimes
        self.seq_len = seq_len
        self.pred_len = pred_len
        
        # Create valid indices
        self.valid_indices = []
        for i in range(len(features) - seq_len - pred_len + 1):
            # Get regime of prediction period (last day of input)
            regime = regimes[i + seq_len - 1]
            if filter_regime is None or regime == filter_regime:
                self.valid_indices.append(i)
    
    def __len__(self):
        return len(self.valid_indices)
    
    def __getitem__(self, idx):
        i = self.valid_indices[idx]
        x = self.features[i: i + self.seq_len]
        y = self.target[i + self.seq_len: i + self.seq_len + self.pred_len]
        return torch.from_numpy(x), torch.from_numpy(y)

In [ ]:
class MultiDLinear(nn.Module):
    """Multivariate DLinear with trend-seasonal decomposition"""
    def __init__(self, seq_len, pred_len, n_features, close_idx=3):
        super().__init__()
        self.seq_len = seq_len
        self.close_idx = close_idx
        self.kernel_size = max(3, seq_len // 4)
        self.fc_trend = nn.Linear(seq_len * n_features, pred_len)
        self.fc_seasonal = nn.Linear(seq_len * n_features, pred_len)
        
    def forward(self, x):
        # x: (B, seq_len, n_features)
        B = x.size(0)
        
        # Decompose close feature
        close_seq = x[:, :, self.close_idx]  # (B, seq_len)
        trend = close_seq.unfold(-1, self.kernel_size, 1).mean(-1)
        pad_left = (self.seq_len - trend.size(-1)) // 2
        pad_right = self.seq_len - trend.size(-1) - pad_left
        trend = F.pad(trend, (pad_left, pad_right), mode='replicate')
        
        # Create trend/seasonal versions
        seasonal = close_seq - trend
        x_trend = x.clone()
        x_seasonal = x.clone()
        x_trend[:, :, self.close_idx] = trend
        x_seasonal[:, :, self.close_idx] = seasonal
        
        return self.fc_trend(x_trend.reshape(B, -1)) + self.fc_seasonal(x_seasonal.reshape(B, -1))

In [ ]:
class EarlyStopping:
    def __init__(self, patience=80):
        self.patience = patience
        self.best_val = float('inf')
        self.wait = 0
        self.best_state = None

    def step(self, val_loss, model):
        if val_loss < self.best_val - 1e-5:
            self.best_val = val_loss
            self.wait = 0
            self.best_state = deepcopy(model.state_dict())
        else:
            self.wait += 1
        return self.wait >= self.patience


def train_model(model, train_loader, val_loader, epochs, lr, patience, device, verbose=True):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=30)
    stopper = EarlyStopping(patience)
    model.to(device)
    
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0
        for bx, by in train_loader:
            bx, by = bx.to(device), by.to(device)
            optimizer.zero_grad()
            pred = model(bx)
            loss = criterion(pred, by)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for bx, by in val_loader:
                bx, by = bx.to(device), by.to(device)
                val_loss += criterion(model(bx), by).item()
        val_loss /= len(val_loader)
        
        scheduler.step(val_loss)
        if stopper.step(val_loss, model):
            if verbose:
                print(f"    Early stopping at epoch {epoch}")
            break
    
    if stopper.best_state:
        model.load_state_dict(stopper.best_state)
    return model

## 6. Train Regime-Specific Models

In [ ]:
# Config
SEQ_LEN = 7  # Input sequence length
PRED_LEN = 100  # Prediction horizon
BATCH_SIZE = 32
NUM_EPOCHS = 500
PATIENCE = 80
LR = 1e-3
# TRAIN_RATIO = 0.7
VAL_RATIO = 0.15

# Get regimes array
regimes = df['regime'].values
target = multi_scaled[:, close_idx]  # close_log scaled

print(f"SEQ_LEN: {SEQ_LEN}")
print(f"PRED_LEN: {PRED_LEN}")

In [ ]:
# Train a model for each regime
regime_models = {}

print("="*60)
print("TRAINING REGIME-SPECIFIC MODELS")
print("="*60)

for regime in sorted_regimes:
    name = REGIME_NAMES[regime]
    print(f"\n▶ Training model for {name} (Regime {regime})...")
    
    # Create dataset filtered by regime
    dataset = MultivariateDataset(
        multi_scaled, target, regimes, 
        SEQ_LEN, PRED_LEN, filter_regime=regime
    )
    
    if len(dataset) < 10:
        print(f"    ⚠️ Not enough samples ({len(dataset)}), skipping...")
        continue
    
    # Split
    total = len(dataset)
    val_len = int(total * VAL_RATIO)
    train_len = total - val_len 
    
    train_subset = torch.utils.data.Subset(dataset, list(range(train_len)))
    val_subset = torch.utils.data.Subset(dataset, list(range(train_len, train_len + val_len)))
    
    if len(val_subset) == 0:
        print(f"    ⚠️ Not enough validation samples, using last 20%...")
        train_len = int(total * 0.8)
        train_subset = torch.utils.data.Subset(dataset, list(range(train_len)))
        val_subset = torch.utils.data.Subset(dataset, list(range(train_len, total)))
    
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)
    
    print(f"    Samples: {len(dataset)} (train={len(train_subset)}, val={len(val_subset)})")
    
    # Create and train model
    model = MultiDLinear(SEQ_LEN, PRED_LEN, n_features, close_idx)
    model = train_model(model, train_loader, val_loader, NUM_EPOCHS, LR, PATIENCE, device)
    
    regime_models[regime] = model
    print(f"    ✅ Model trained!")

print(f"\nTotal models trained: {len(regime_models)}")

In [ ]:
# Also train a global model (all data)
print("\n▶ Training GLOBAL model (all regimes)...")

dataset_all = MultivariateDataset(
    multi_scaled, target, regimes,
    SEQ_LEN, PRED_LEN, filter_regime=None
)

total = len(dataset_all)
train_len = int(total * 0.9)
train_subset = torch.utils.data.Subset(dataset_all, list(range(train_len)))
val_subset = torch.utils.data.Subset(dataset_all, list(range(train_len, total)))

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE)

print(f"    Samples: {len(dataset_all)} (train={len(train_subset)}, val={len(val_subset)})")

global_model = MultiDLinear(SEQ_LEN, PRED_LEN, n_features, close_idx)
global_model = train_model(global_model, train_loader, val_loader, NUM_EPOCHS, LR, PATIENCE, device)
print("    ✅ Global model trained!")

## 7. Generate Predictions

In [ ]:
def predict_with_model(model, input_data, device):
    """Generate prediction from input data"""
    model.eval()
    with torch.no_grad():
        x = torch.from_numpy(input_data.astype(np.float32)).unsqueeze(0).to(device)
        pred = model(x).cpu().numpy().flatten()
    return pred


def inverse_transform_prediction(pred_scaled, scaler):
    """Convert scaled prediction back to price"""
    pred_log = scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
    pred_price = np.exp(pred_log)
    return pred_price

In [ ]:
# Get current regime (last day)
current_regime = df['regime'].iloc[-1]
current_regime_name = REGIME_NAMES[current_regime]

print(f"Current market regime: {current_regime_name} (Regime {current_regime})")

# Get last SEQ_LEN data points
input_data = multi_scaled[-SEQ_LEN:].copy()

print(f"Input shape: {input_data.shape}")

In [ ]:
# Strategy 1: Use regime-specific model
if current_regime in regime_models:
    pred_regime = predict_with_model(regime_models[current_regime], input_data, device)
    prices_regime = inverse_transform_prediction(pred_regime, uni_scaler)
    print(f"\nRegime-specific prediction ({current_regime_name}):")
    print(f"  Range: {prices_regime.min():.2f} - {prices_regime.max():.2f}")
else:
    print(f"\n⚠️ No specific model for {current_regime_name}, using global model")
    prices_regime = None

# Strategy 2: Use global model
pred_global = predict_with_model(global_model, input_data, device)
prices_global = inverse_transform_prediction(pred_global, uni_scaler)
print(f"\nGlobal model prediction:")
print(f"  Range: {prices_global.min():.2f} - {prices_global.max():.2f}")

In [ ]:
# Strategy 3: Weighted ensemble based on regime confidence
# Get regime probabilities for current state
current_features = df_regime[regime_features].iloc[-1:].values
current_features_scaled = regime_scaler.transform(current_features)
regime_probs = hmm_model.predict_proba(current_features_scaled)[0]

print("\nRegime probabilities:")
for r in sorted_regimes:
    print(f"  {REGIME_NAMES[r]}: {regime_probs[r]*100:.1f}%")

# Weighted ensemble
ensemble_pred = np.zeros(PRED_LEN)
total_weight = 0

for regime, model in regime_models.items():
    weight = regime_probs[regime]
    pred = predict_with_model(model, input_data, device)
    ensemble_pred += weight * pred
    total_weight += weight

# Add global model with remaining weight
global_weight = max(0, 1 - total_weight)
if global_weight > 0:
    ensemble_pred += global_weight * pred_global

prices_ensemble = inverse_transform_prediction(ensemble_pred, uni_scaler)
print(f"\nEnsemble prediction (weighted by regime probability):")
print(f"  Range: {prices_ensemble.min():.2f} - {prices_ensemble.max():.2f}")

In [ ]:
# Choose final prediction
# Use ensemble as primary, fallback to global if needed
final_prices = prices_ensemble

print("\n" + "="*60)
print("FINAL PREDICTION")
print("="*60)
print(f"Method: Weighted Ensemble")
print(f"Range: {final_prices.min():.2f} - {final_prices.max():.2f} VND")
print(f"First 5 days: {final_prices[:5]}")
print(f"Last 5 days: {final_prices[-5:]}")

## 8. Create Submission & Visualization

In [ ]:
# Create submission
os.makedirs('submissions', exist_ok=True)

submission = pd.DataFrame({
    'id': np.arange(1, PRED_LEN + 1),
    'close': final_prices
})

submission_path = f'submissions/submission_hmm_regime_multidlinear_{SEQ_LEN}d.csv'
submission.to_csv(submission_path, index=False)

print(f"\nSubmission saved to: {submission_path}")
print(f"\nPreview:")
display(submission.head(10))
display(submission.tail(5))

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Plot 1: Historical + Forecast
ax = axes[0]
hist_days = 200
hist_dates = df['time'].iloc[-hist_days:]
hist_prices = df['close'].iloc[-hist_days:]

last_date = df['time'].iloc[-1]
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=PRED_LEN, freq='B')

ax.plot(hist_dates, hist_prices, 'b-', linewidth=2, label='Historical')
ax.plot(future_dates, final_prices, 'r--', linewidth=2, label='Forecast (Ensemble)')

# Also show individual predictions
if prices_regime is not None:
    ax.plot(future_dates, prices_regime, 'g:', linewidth=1, alpha=0.7, 
            label=f'Regime-specific ({current_regime_name})')
ax.plot(future_dates, prices_global, 'orange', linestyle=':', linewidth=1, alpha=0.7,
        label='Global model')

ax.axvline(last_date, color='gray', linestyle='--', alpha=0.7)
ax.set_xlabel('Date')
ax.set_ylabel('Close Price (VND)')
ax.set_title(f'FPT Stock Price Forecast - HMM Regime Clustering + MultiDLinear', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Comparison of methods
ax = axes[1]
ax.plot(range(1, PRED_LEN+1), final_prices, 'r-', linewidth=2, label='Ensemble (Final)')
if prices_regime is not None:
    ax.plot(range(1, PRED_LEN+1), prices_regime, 'g--', linewidth=1.5, 
            label=f'Regime: {current_regime_name}')
ax.plot(range(1, PRED_LEN+1), prices_global, 'b--', linewidth=1.5, label='Global')

ax.set_xlabel('Day')
ax.set_ylabel('Predicted Price (VND)')
ax.set_title('Comparison of Prediction Methods', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('submissions/forecast_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Visualization saved to: submissions/forecast_visualization.png")

## 9. Summary

In [ ]:
print("\n" + "="*60)
print("📊 SUMMARY")
print("="*60)
print(f"\n🔍 Market Regime Detection:")
print(f"   Method: HMM with 60-day window")
print(f"   Regimes: {N_REGIMES}")
for r in sorted_regimes:
    count = (df['regime'] == r).sum()
    pct = count / len(df) * 100
    print(f"   - {REGIME_NAMES[r]}: {pct:.1f}%")

print(f"\n📈 Current State:")
print(f"   Regime: {current_regime_name}")
print(f"   Last Price: {df['close'].iloc[-1]:.2f} VND")

print(f"\n🎯 Prediction:")
print(f"   Method: Weighted Ensemble")
print(f"   Horizon: {PRED_LEN} days")
print(f"   Price Range: {final_prices.min():.2f} - {final_prices.max():.2f} VND")
pct_change = (final_prices[-1] / df['close'].iloc[-1] - 1) * 100
print(f"   Expected Change: {pct_change:+.1f}%")

print(f"\n📁 Output:")
print(f"   Submission: {submission_path}")
print("\n" + "="*60)